# A Journey through the Evolution of Deep Learning and the Quest for Human-Level Control via Deep Reinforcement Learning

### By Atanas Vasev

# 1. Introduction

Despite the seemingly futuristic and complex nature of the term 'Deep Learning,' its roots can be traced back to 1940 with the pioneering work of Warren McCulloch and Walter Pitts. They laid the foundation for neural networks with their influential paper titled 'A Logical Calculus of Ideas Immanent in Nervous Activity.' However, progress in neural networks faced challenges for decades due to limitations in computing power and data availability. It wasn't until the early 21st century that significant advancements reignited interest, with a transformative moment occurring in 2012 during the ImageNet Large Scale Visual Recognition Challenge.

This pivotal moment was characterized by the breakthrough achieved by a team led by Alex Krizhevsky, Ilya Sutskever, and Geoffrey Hinton. They demonstrated a groundbreaking leap in image classification accuracy using a deep neural network architecture known as AlexNet. This success marked the resurgence of deep learning, showcasing its superior capabilities in handling complex tasks.

The success of AlexNet opened the door for extensive exploration and development in deep neural networks. Researchers delved into designing more sophisticated architectures, introducing techniques like convolutional layers for image processing. This momentum catalyzed breakthroughs in various domains, including natural language processing and speech recognition.

The landscape of deep learning further evolved with the introduction of reinforcement learning techniques, notably highlighted by DeepMind's Deep Q-Learning algorithm. In particular, Deep Q-Learning garnered attention with its remarkable performance in the game of Breakout. DeepMind demonstrated the ability of the algorithm to achieve superhuman performance by learning optimal strategies through reinforcement learning.

As researchers progressed, they continued to refine and expand reinforcement learning methodologies. The focus on enhancing training techniques and algorithmic advancements led to further breakthroughs, making reinforcement learning a key player in the evolution of deep learning.

The dynamic journey of deep learning, from the foundational work of McCulloch and Pitts to the complex, versatile neural networks of today, is exemplified by the continuous exploration of novel architectures, training methodologies, and applications. In the realm of reinforcement learning, the impact of algorithms like Deep Q-Learning serves as a testament to the ongoing commitment to unraveling the full potential of deep learning in replicating and surpassing human-level control, particularly in challenging domains such as Breakout.

In this paper, I will attempt to replicate DeepMind's Deep Q-Learning algorithm. The primary objective is to achieve a greater understanding not only of reinforcement learning but also of the broader topic of deep learning. The reason I chose this particular paper and topic is that this type of deep learning architecture can serve as a valuable case study for its success in achieving human-level performance in complex tasks. Deep Q-Learning, proposed by researchers at DeepMind, has demonstrated remarkable capabilities in mastering complex environments through reinforcement learning. By replicating and dissecting the nuances of this algorithm, I aim to unravel insights into the principles of reinforcement learning and the intricate mechanisms that contribute to the success of deep learning models.

The journey involves implementing, testing, and fine-tuning the Deep Q-Learning algorithm, aiming not only for successful replication but also for a deeper comprehension of the underlying principles. Furthermore, this endeavor provides an opportunity to explore the broader implications of deep learning, such as its adaptability to different domains and its potential for scalable and efficient learning.

As the paper progresses, I will delve into the historical context of deep learning, tracing its evolution from its early theoretical foundations to the recent breakthroughs in algorithmic advancements. By synthesizing this historical narrative with practical implementation, the aim is to provide a comprehensive exploration of deep learning, emphasizing its transformative impact on artificial intelligence.

Through the lens of replicating Deep Q-Learning, this paper seeks to contribute to the ongoing discourse on the challenges, advancements, and future prospects of deep learning. By gaining insights into the inner workings of this algorithm, we can further our understanding of the broader implications of reinforcement learning and its potential applications in real-world scenarios.

In 2013 a London based startup called DeepMind published a groundbreaking paper called ["Playing Atari with Deep Reinforcement Learning"](https://arxiv.org/abs/1312.5602) on arXiv: The authors presented a variant of Reinforcement Learning called Deep Q-Learning that is able to successfully learn control policies for different Atari 2600 games receiving only screen pixels as input and a reward when the game score changes. The agent even surpasses human expert players in some of those games! This is an astonishing result because previously “AIs” used to be limited to one single game, for instance, chess, whereas in this case the types and contents of the games in the Arcade Learning Environment vary significantly and yet no adjustment of the architecture, learning algorithm or hyperparameters is needed.

No wonder DeepMind was bought by Google for 500 Million Dollars. The company has since been one of the leading institutions advancing Deep Learning research and a later article discussing DQN(Deep Q-Networks) has been published in Nature

# 2. Deep Q-Learning

Before we start the implementation we need some basics